# Campeonato Brasileiro Série B 2020 - Jogos


Análise do campeonato, com base nos jogos.

Os dados foram extraídos do site da CBF.

https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b/2020

In [1]:
# Bibliotecas

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import json
import requests
import re
from bs4 import BeautifulSoup
from datetime import datetime
from os.path import exists

%load_ext autoreload
%autoreload 2

from ExtrairJogos import ExtrairJogos
from funcoes import criar_dataframe

### Carregar dados

In [2]:
arquivo_dados = 'tabela_jogos_serie_b_2020.csv'

link_pagina = 'https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b/2020'

In [3]:
if not exists('data/' + arquivo_dados):
    pagina = ExtrairJogos.extrair_pagina(link_pagina)
    blocos = ExtrairJogos.extrair_area_dados(pagina)
    df_tabela_jogos = criar_dataframe(blocos)
    df_tabela_jogos.to_csv('data/' + arquivo_dados, sep=';', encoding='utf-8', index=False)
else:
    df_tabela_jogos = pd.read_csv('data/' + arquivo_dados, delimiter=';', encoding='utf-8')

In [4]:
df_tabela_jogos

,Rodada,Data,Time casa,Gols time casa,Time visitante,Gols time visitante
0,1,07/08/2020,Cuiabá - MT,0,Brasil - RS,0
1,1,07/08/2020,Confiança - SE,2,Paraná - PR,2
2,1,08/08/2020,Juventude - RS,2,Crb - AL,1
3,1,08/08/2020,Operário - PR,3,Figueirense - SC,1
4,1,08/08/2020,Avaí - SC,3,Náutico - PE,1
...,...,...,...,...,...,...
375,38,29/01/2021,America - MG,2,Avaí - SC,1
376,38,29/01/2021,Brasil - RS,0,Vitória - BA,1
377,38,29/01/2021,Paraná - PR,0,Cruzeiro - MG,0
378,38,29/01/2021,Guarani - SP,0,Juventude - RS,1


### Tratar dados

Eu particularmente não gosto de ver os nomes dos times escritos com os estados, exceto nos clubes mais conhecidos que tenham nomes repetidos, como os Américas e Atléticos (entre outros), onde aí sim, acho necessário distinção.

In [5]:
df_tabela_jogos['Time casa'].unique()

array(['Cuiabá - MT', 'Confiança - SE', 'Juventude - RS', 'Operário - PR',
       'Avaí - SC', 'Vitória - BA', 'Cruzeiro - MG', 'Ponte Preta - SP',
       'Csa - AL', 'Oeste - SP', 'Botafogo - SP', 'Brasil - RS',
       'Sampaio Correa - MA', 'Guarani - SP', 'Paraná - PR',
       'America - MG', 'Figueirense - SC', 'Náutico - PE', 'Crb - AL',
       'Chapecoense - SC'], dtype=object)

Para isso, criei um arquivo `json` mapeando os nomes das equipes que vem na base de dados, e os nomes que eu sugeri para alteração.

In [6]:
with open('data/times_map.json', encoding='utf-8') as json_file:
    times_map = json.load(json_file)

times_map

{'Cuiabá - MT': 'Cuiabá',
 'Confiança - SE': 'Confiança',
 'Juventude - RS': 'Juventude',
 'Operário - PR': 'Operário',
 'Avaí - SC': 'Avaí',
 'Vitória - BA': 'Vitória',
 'Cruzeiro - MG': 'Cruzeiro',
 'Ponte Preta - SP': 'Ponte Preta',
 'Csa - AL': 'CSA',
 'Oeste - SP': 'Oeste',
 'Botafogo - SP': 'Botafogo-SP',
 'Brasil - RS': 'Brasil-RS',
 'Sampaio Correa - MA': 'Sampaio Correa',
 'Guarani - SP': 'Guarani',
 'Paraná - PR': 'Paraná',
 'America - MG': 'America-MG',
 'Figueirense - SC': 'Figueirense',
 'Náutico - PE': 'Náutico',
 'Crb - AL': 'CRB',
 'Chapecoense - SC': 'Chapecoense'}

In [7]:
# Fazendo o re-map

df_tabela_jogos = df_tabela_jogos.replace({'Time casa': times_map})
df_tabela_jogos = df_tabela_jogos.replace({'Time visitante': times_map})

In [8]:
df_tabela_jogos['Time casa'].unique()

array(['Cuiabá', 'Confiança', 'Juventude', 'Operário', 'Avaí', 'Vitória',
       'Cruzeiro', 'Ponte Preta', 'CSA', 'Oeste', 'Botafogo-SP',
       'Brasil-RS', 'Sampaio Correa', 'Guarani', 'Paraná', 'America-MG',
       'Figueirense', 'Náutico', 'CRB', 'Chapecoense'], dtype=object)

In [9]:
df_tabela_jogos['Time visitante'].unique()

array(['Brasil-RS', 'Paraná', 'CRB', 'Figueirense', 'Náutico',
       'Sampaio Correa', 'Botafogo-SP', 'America-MG', 'Guarani',
       'Chapecoense', 'Confiança', 'Ponte Preta', 'Juventude', 'Cruzeiro',
       'Avaí', 'Cuiabá', 'Vitória', 'Operário', 'Oeste', 'CSA'],
      dtype=object)

### Criar tabela dinâmica

Apesar do termo evocar o excel, a ideia aqui é um pouco diferente.

O objetivo é construir um dataframe que irá conter o desempenho das equipes a cada rodada que passa. Na rodada 1, será calculado a pontuação das equipes, a posição que se encontra no campeonato, se o time ganhou, empatou ou perdeu, o aproveitamento de pontos, etc. Na segunda rodada, será feito o mesmo cálculo, e, um acumulado com a primeira rodada será salvo, e assim por diante, rodada por rodada.

Ao final, será possível traçar a trajetória de cada equipe.

Essa tabela terá as colunas:

- Rodada
- Posição
- Equipe
- Pontos (geral, casa, visitante)
- Jogos (geral, casa, visitante)
- Vitorias (geral, casa, visitante)
- Empates (geral, casa, visitante)
- Derrotas (geral, casa, visitante)
- Gols Pró (geral, casa, visitante)
- Gols Contra (geral, casa, visitante)
- Saldo de Gols (geral, casa, visitante)
- Aproveitamento (geral, casa, visitante)


O primeiro passo, é criar um dataframe com os nomes das colunas, os nomes das equipes, já setando 1 para rodada e posição. Os demais valores deverão todos estar zerados, assim teremos o estado inicial da tabela, antes dos jogos começarem.

In [10]:
colunas = ['rodada', 'equipe', 'pontos',  'jogos', 'vitorias', 'empates', 'derrotas', 
           'gols_pro', 'gols_contra', 'saldo_gols',  'aproveitamento', 'pontos_casa', 'jogos_casa', 'vitorias_casa', 
           'empates_casa', 'derrotas_casa', 'gols_pro_casa', 'gols_contra_casa', 
           'saldo_gols_casa', 'aproveitamento_casa', 'pontos_visitante', 'jogos_visitante', 
           'vitorias_visitante', 'empates_visitante', 'derrotas_visitante', 
           'gols_pro_visitante', 'saldo_gols_visitante', 'gols_contra_visitante',
           'aproveitamento_visitante']

In [11]:
# Funções

def criar_dataframe_padrao(colunas, equipes):
    
    df = pd.DataFrame(columns=colunas)
    df['equipe'] = equipes
    df['rodada'] = 0
    df = df.fillna(0) # preencher com zeros onde está NaN
    
    return df

def calcular_pontos(a, b):
    if a == b:
        return 1
    elif a > b:
        return 3
    else: 
        return 0
    
def checar_vitoria(a, b):
    if a > b:
        return 1
    else:
        return 0

def checar_empate(a, b):
    if a == b:
        return 1
    else:
        return 0

def checar_derrota(a, b):
    if a < b:
        return 1
    else:
        return 0
    
def validar_valores(valor):
    if len(valor.values) > 0:
        return valor.values[0]
    else:
        return 0

**A punição do Cruzeiro**

Não podemos esquecer que este campeonato possui algo atípico: o Cruzeiro que fora rebaixado no ano anterior, foi punido por conta de dívida não paga, com menos 6 pontos. Essa punição foi dada antes do início da competição, e o clube começou com -6 pontos. 

Na tabela, isso será refletido apenas no campo `pontos`. Se houver a tentativa de calcular pontos com os demais campos, vai ter inconsistência com a quantidade final de pontos.

In [107]:
# Zerar dataframe
df_serieb_dinamica = criar_dataframe_padrao(colunas, df_tabela_jogos['Time casa'].unique())

# Caso do Cruzeiro *
li = df_serieb_dinamica[(df_serieb_dinamica['equipe'] == 'Cruzeiro') & (df_serieb_dinamica['rodada'] == 0)][['pontos']]
df_serieb_dinamica.loc[li.index.values[0], li.columns.values[0]] = -6

In [102]:
df_serieb_dinamica

,rodada,equipe,pontos,jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo_gols,...,aproveitamento_casa,pontos_visitante,jogos_visitante,vitorias_visitante,empates_visitante,derrotas_visitante,gols_pro_visitante,saldo_gols_visitante,gols_contra_visitante,aproveitamento_visitante
0,0,Cuiabá,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,Confiança,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,Juventude,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,Operário,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,Avaí,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,Vitória,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,Cruzeiro,-6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,Ponte Preta,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,CSA,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,Oeste,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [110]:
df_serieb_dinamica.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 780 entries, 0 to 779
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rodada                    780 non-null    int64  
 1   equipe                    780 non-null    object 
 2   pontos                    780 non-null    int64  
 3   jogos                     780 non-null    int64  
 4   vitorias                  780 non-null    int64  
 5   empates                   780 non-null    int64  
 6   derrotas                  780 non-null    int64  
 7   gols_pro                  780 non-null    int64  
 8   gols_contra               20 non-null     float64
 9   saldo_gols                20 non-null     float64
 10  aproveitamento            20 non-null     float64
 11  pontos_casa               20 non-null     float64
 12  jogos_casa                20 non-null     float64
 13  vitorias_casa             20 non-null     float64
 14  empates_ca

In [116]:
# Zerar dataframe
df_serieb_dinamica = criar_dataframe_padrao(colunas, df_tabela_jogos['Time casa'].unique())

# Caso do Cruzeiro *
li = df_serieb_dinamica[(df_serieb_dinamica['equipe'] == 'Cruzeiro') & (df_serieb_dinamica['rodada'] == 0)][['pontos']]
df_serieb_dinamica.loc[li.index.values[0], li.columns.values[0]] = -6

for rodada in np.arange(1,39):
    jogos_rodada = df_tabela_jogos.loc[(df_tabela_jogos['Rodada'] == rodada)]
    
    rodadas = []
    equipes = []
    pontos = []
    jogos = []
    vitorias = []
    empates = []
    derrotas = []
    gols_pro = []
    gols_contra = []
    saldo_gols = []
    
    for idx, data in jogos_rodada.iterrows():
        row_dict = {}
        
        #####################################################################
        # Time casa
        #####################################################################
        
        rodada_anterior = df_serieb_dinamica.loc[
            (df_serieb_dinamica['rodada'] == rodada - 1) & (df_serieb_dinamica['equipe'] == data['Time casa'])]
        
        # Atualizar tabela
        rodadas.append(rodada)
        jogos.append(rodada)
        equipes.append(data['Time casa'])
        
        pontos.append(calcular_pontos(data['Gols time casa'], data['Gols time visitante']) + validar_valores(rodada_anterior['pontos']))
        
        vitorias.append(checar_vitoria(data['Gols time casa'], data['Gols time visitante']) +
                        validar_valores(rodada_anterior['vitorias'])
                       )
        
        empates.append(checar_empate(data['Gols time casa'], data['Gols time visitante']) + 
                      validar_valores(rodada_anterior['empates'])
                      )
        
        derrotas.append(checar_derrota(data['Gols time casa'], data['Gols time visitante']) + 
                       validar_valores(rodada_anterior['derrotas']))
        
        gols_pro.append(data['Gols time casa'] + validar_valores(rodada_anterior['gols_pro']))
        gols_contra.append(data['Gols time visitante'] + validar_valores(rodada_anterior['gols_contra']))
        
        # saldo_gols
        
        #####################################################################
        # Time visitante
        #####################################################################
        
        rodada_anterior = df_serieb_dinamica.loc[
            (df_serieb_dinamica['rodada'] == rodada - 1) & (df_serieb_dinamica['equipe'] == data['Time visitante'])]
        
        # Atualizar tabela
        rodadas.append(rodada)
        jogos.append(rodada)
        equipes.append(data['Time visitante'])
        
        pontos.append(calcular_pontos(data['Gols time visitante'], data['Gols time casa']) + 
                      validar_valores(rodada_anterior['pontos']))
        
        vitorias.append(checar_vitoria(data['Gols time visitante'], data['Gols time casa']) + 
                        validar_valores(rodada_anterior['vitorias']))
        
        empates.append(checar_empate(data['Gols time visitante'], data['Gols time casa']) + 
                       validar_valores(rodada_anterior['empates']))

        derrotas.append(checar_derrota(data['Gols time visitante'], data['Gols time casa']) +
                        validar_valores(rodada_anterior['derrotas']))
        
        gols_pro.append(data['Gols time visitante'] + validar_valores(rodada_anterior['gols_pro']))
        gols_contra.append(data['Gols time casa'] + validar_valores(rodada_anterior['gols_contra']))
    
    ################################################
    # Montar dataframe final
    ################################################
    
    row_dict = {
        'rodada': rodadas,
        'equipe': equipes,
        'pontos': pontos,
        'jogos': jogos,
        'vitorias': vitorias,
        'empates': empates,
        'derrotas': derrotas,
        'gols_pro': gols_pro,
        'gols_contra': gols_contra
    }
    
    # Criar um dataframe com as infos da rodada
    _df = pd.DataFrame.from_dict(row_dict)
    _df = _df.fillna(0)
    _df.index = np.arange(1, len(_df) + 1)
    _df = _df.sort_values(by=['pontos', 'vitorias', 'gols_pro'], ascending=[False, False, False], ignore_index=True).reset_index()
    _df.rename(columns = {'index': 'posicao'}, inplace=True)
    _df['posicao'] += 1
    
    # Atualizar dataframe principal
    df_serieb_dinamica = pd.concat([df_serieb_dinamica, _df], ignore_index=True)

**Tabela consolidada**

In [117]:
df_serieb_dinamica.loc[df_serieb_dinamica['rodada'] == 38]

,rodada,equipe,pontos,jogos,vitorias,empates,derrotas,gols_pro,gols_contra,saldo_gols,...,pontos_visitante,jogos_visitante,vitorias_visitante,empates_visitante,derrotas_visitante,gols_pro_visitante,saldo_gols_visitante,gols_contra_visitante,aproveitamento_visitante,posicao
760,38,America-MG,73,38,20,13,5,43,23,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
761,38,Chapecoense,73,38,20,13,5,42,21,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
762,38,Juventude,61,38,17,10,11,52,42,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
763,38,Cuiabá,61,38,17,10,11,48,40,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
764,38,CSA,58,38,16,10,12,50,37,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0
765,38,Sampaio Correa,57,38,17,6,15,50,38,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
766,38,Ponte Preta,57,38,16,9,13,54,49,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0
767,38,Operário,57,38,15,12,11,40,34,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
768,38,Avaí,55,38,16,7,15,45,49,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
769,38,CRB,52,38,15,7,16,48,47,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0


---

### Referências

https://ge.globo.com/futebol/times/cruzeiro/noticia/cruzeiro-chega-a-terceira-punicao-em-2020-e-ve-reflexo-de-contas-nao-pagas-no-campo-outra-vez.ghtml

https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-b/2020